In [1]:
import numpy as np
import lasagne as L
import time

import sys
sys.path.append('../')
sys.path.append('../../')
from HRED import HRED

from data_load.redditv3_load import load_pairs

reddit_path = "/pio/data/data/reddit_sample/v3/"

train_pairs, test_pairs = load_pairs(path=reddit_path)

glove6B = np.load('/pio/data/data/glove_vec/6B/glove/glove.6B.300d.npy')

idx_to_w = np.load('/pio/data/data/glove_vec/6B/glove/glove.6B.wordlist.pkl')
voc_size = len(idx_to_w)
w_to_idx = {idx_to_w[i] : i for i in xrange(voc_size)}

###

net = HRED(voc_size=voc_size,
           emb_size=300,
           lv1_rec_size=300,
           lv2_rec_size=300,
           out_emb_size=300,
           num_sampled=1000,
           emb_init=glove6B,
           train_emb=False,
           train_inds=[0, 400002, 400003],
           skip_gen=True)


# net = HRED(voc_size=200000,
#            emb_size=300,
#            lv1_rec_size=300,
#            lv2_rec_size=300,
#            out_emb_size=300,
#            num_sampled=1000,
#            emb_init=None,
#            train_emb=True,
#            # train_inds=[0, 400002, 400003],
#            skip_gen=True)

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is disabled, cuDNN 4007)


Building the model...
Compiling theano functions...
    train_fn...
    val_fn...
Skipping generating part...
Done


In [14]:
params = L.layers.get_all_params(net.train_net, trainable=True)
all_params = L.layers.get_all_params(net.train_net)

In [3]:
params

[emb.W,
 GRU1forw.W_in_to_updategate,
 GRU1forw.W_hid_to_updategate,
 GRU1forw.b_updategate,
 GRU1forw.W_in_to_resetgate,
 GRU1forw.W_hid_to_resetgate,
 GRU1forw.b_resetgate,
 GRU1forw.W_in_to_hidden_update,
 GRU1forw.W_hid_to_hidden_update,
 GRU1forw.b_hidden_update,
 GRU1back.W_in_to_updategate,
 GRU1back.W_hid_to_updategate,
 GRU1back.b_updategate,
 GRU1back.W_in_to_resetgate,
 GRU1back.W_hid_to_resetgate,
 GRU1back.b_resetgate,
 GRU1back.W_in_to_hidden_update,
 GRU1back.W_hid_to_hidden_update,
 GRU1back.b_hidden_update,
 GRU2.W_in_to_updategate,
 GRU2.W_hid_to_updategate,
 GRU2.b_updategate,
 GRU2.W_in_to_resetgate,
 GRU2.W_hid_to_resetgate,
 GRU2.b_resetgate,
 GRU2.W_in_to_hidden_update,
 GRU2.W_hid_to_hidden_update,
 GRU2.b_hidden_update,
 dec_init.W,
 dec_init.b,
 GRUdec.W_in_to_updategate,
 GRUdec.W_hid_to_updategate,
 GRUdec.b_updategate,
 GRUdec.W_in_to_resetgate,
 GRUdec.W_hid_to_resetgate,
 GRUdec.b_resetgate,
 GRUdec.W_in_to_hidden_update,
 GRUdec.W_hid_to_hidden_update,
 

In [15]:
for p in all_params:
    print p.shape.eval(), '      ', p

[400004    300]        emb.E
[  3 300]        emb.W
[300 300]        GRU1forw.W_in_to_updategate
[300 300]        GRU1forw.W_hid_to_updategate
[300]        GRU1forw.b_updategate
[300 300]        GRU1forw.W_in_to_resetgate
[300 300]        GRU1forw.W_hid_to_resetgate
[300]        GRU1forw.b_resetgate
[300 300]        GRU1forw.W_in_to_hidden_update
[300 300]        GRU1forw.W_hid_to_hidden_update
[300]        GRU1forw.b_hidden_update
[  1 300]        GRU1forw.hid_init
[300 300]        GRU1back.W_in_to_updategate
[300 300]        GRU1back.W_hid_to_updategate
[300]        GRU1back.b_updategate
[300 300]        GRU1back.W_in_to_resetgate
[300 300]        GRU1back.W_hid_to_resetgate
[300]        GRU1back.b_resetgate
[300 300]        GRU1back.W_in_to_hidden_update
[300 300]        GRU1back.W_hid_to_hidden_update
[300]        GRU1back.b_hidden_update
[  1 300]        GRU1back.hid_init
[600 300]        GRU2.W_in_to_updategate
[300 300]        GRU2.W_hid_to_updategate
[300]        GRU2.b_updateg

In [19]:
a = net.iterate_minibatches(train_pairs, 150).next()

In [23]:
a[0].shape

(150, 2, 19)

In [24]:
last_scores = [np.inf]
max_epochs_wo_improvement = 5
tol = 0.001
epoch = 1
best_epoch = None

model_filename = 'trained_models/test1/redditv3_pairs_gloveFixed_bs25_early5'

t0 = time.time()
while len(last_scores) <= max_epochs_wo_improvement or last_scores[0] > min(last_scores) + tol:
    print '\n\nStarting epoch {}...\n'.format(epoch)
    train_error = net.train_one_epoch(train_data=train_pairs, batch_size=1500, log_interval=1)
    val_error = net.validate(val_data=test_pairs, batch_size=1500)
    print '\nTraining loss:   {}'.format(train_error)
    print 'Validation loss: {}'.format(val_error)

    if val_error < min(last_scores):
        print '\nSaving model...'
        net.save_params(model_filename + '_ep' + str(epoch) + '.npz')
        print 'Done saving.'
        best_epoch = epoch

    last_scores.append(val_error)

    if len(last_scores) > max_epochs_wo_improvement + 1:
        del last_scores[0]

    epoch += 1


print '\n\nTotal training time: {:.2f}s'.format(time.time() - t0)
print 'Best model after {} epochs with loss {}'.format(best_epoch, min(last_scores))
print 'Validation set perplexity: {}'.format(np.exp(min(last_scores)))
print 'Model saved as ' + model_filename



Starting epoch 1...



MemoryError: Error allocating 264000000 bytes of device memory (out of memory).
Apply node that caused the error: GpuDot22(GpuElemwise{Add}[(0, 1)].0, GpuDimShuffle{1,0}.0)
Toposort index: 562
Inputs types: [CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix)]
Inputs shapes: [(66000, 300), (300, 1000)]
Inputs strides: [(300, 1), (1, 300)]
Inputs values: ['not shown', 'not shown']
Inputs type_num: ['', '']
Outputs clients: [[GpuElemwise{add,no_inplace}(GpuDot22.0, GpuDimShuffle{x,0}.0)]]

Debugprint of the apply node: 
GpuDot22 [id A] <CudaNdarrayType(float32, matrix)> ''   
 |GpuElemwise{Add}[(0, 1)] [id B] <CudaNdarrayType(float32, matrix)> ''   
 | |GpuDimShuffle{x,0} [id C] <CudaNdarrayType(float32, row)> ''   
 | | |h0.b [id D] <CudaNdarrayType(float32, vector)>
 | |GpuGemm{inplace} [id E] <CudaNdarrayType(float32, matrix)> ''   
 |   |GpuDot22 [id F] <CudaNdarrayType(float32, matrix)> ''   
 |   | |GpuReshape{2} [id G] <CudaNdarrayType(float32, matrix)> ''   
 |   |

In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

import sys
sys.path.append('../')

from reddit_load import load_singles, load_pairs, get_reddit_voc

%aimport HRED

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
%%time
reddit_path = "/pio/data/data/reddit_sample/"

train_singles, test_singles = load_singles(path=reddit_path)
data_pairs = load_pairs(path=reddit_path)
idx_to_w, w_to_idx, voc_size, freqs = get_reddit_voc(path=reddit_path)

CPU times: user 9.62 s, sys: 109 ms, total: 9.72 s
Wall time: 9.72 s


In [3]:
%%time
hred_net = HRED.HRED(voc_size=voc_size,
                     emb_size=300,
                     lv1_rec_size=300, 
                     lv2_rec_size=300, 
                     out_emb_size=300, 
                     num_sampled=200,
                     ssoft_probs=freqs,
                     n=1)

Building the model...
Compiling theano functions...
Building a network for generating...
Done
CPU times: user 36.5 s, sys: 965 ms, total: 37.5 s
Wall time: 37.4 s


In [6]:
# hred_net.load_params('')

In [5]:
# 2ep

hred_net.train_one_epoch(data_singles, 60, log_interval=100)

Done 100 batches in 5.00s	training loss:	3.428550
Done 200 batches in 9.97s	training loss:	3.421801
Done 300 batches in 14.99s	training loss:	3.422451
Done 400 batches in 19.95s	training loss:	3.414273
Done 500 batches in 24.35s	training loss:	3.395562
Done 600 batches in 28.43s	training loss:	3.354878
Done 700 batches in 32.85s	training loss:	3.345874
Done 800 batches in 37.86s	training loss:	3.360305
Done 900 batches in 42.60s	training loss:	3.361262
Done 1000 batches in 47.48s	training loss:	3.366504
Done 1100 batches in 52.54s	training loss:	3.374706
Done 1200 batches in 57.65s	training loss:	3.379332
Done 1300 batches in 62.66s	training loss:	3.383389
Done 1400 batches in 67.66s	training loss:	3.385690
Done 1500 batches in 72.73s	training loss:	3.387937
Done 1600 batches in 77.82s	training loss:	3.391383
Done 1700 batches in 82.68s	training loss:	3.392958
Done 1800 batches in 85.33s	training loss:	3.362766
Done 1900 batches in 89.99s	training loss:	3.361663
Done 2000 batches in 94

3.3252608175728415

In [ ]:
hred_net.save_params('')